In [1]:
import os
import pandas as pd
import numpy as np


main_directory = os.path.normpath(os.getcwd() + os.sep + os.pardir)
data_directory = os.path.join(main_directory, 'data_cleaning')

player_df = pd.read_csv(os.path.join(data_directory,'cleaned_player_stat_w_MVP_defensive_player.csv'),index_col=0)

player_df.head()

,Player,Age,Position,Games started,Minutes played per game,Field goals per game,Field goals attempt per game,3 point field goal per game,3 point field goal attempt per game,2 point field goal per game,...,Turn overs per game,Personal fouls per game,Points per game,Field goal percent,3 point field goal percentage,2 point field goal percentage,Free throw percentage,Effective field goal percentage,MVP,Defensive Player of the Year
0,A.C. Green,37,PF,1,1411,144,324,0,6,144,...,45,119,367,0.444444,0.000000,0.452830,0.711712,0.444444,0,0
1,A.J. Guyton,22,PG,8,630,78,192,27,69,51,...,24,35,198,0.406250,0.391304,0.414634,0.833333,0.476562,0,0
2,Aaron McKie,28,SG,33,2394,338,714,53,170,285,...,203,178,878,0.473389,0.311765,0.523897,0.768041,0.510504,0,0
3,Aaron Williams,29,PF,25,2336,297,650,0,2,297,...,132,319,838,0.456923,0.000000,0.458333,0.787097,0.456923,0,0
4,Adam Keefe,30,PF,13,836,64,159,1,3,63,...,40,102,168,0.402516,0.333333,0.403846,0.619048,0.405660,0,0


In [2]:
player_df.head()

,Player,Age,Position,Games started,Minutes played per game,Field goals per game,Field goals attempt per game,3 point field goal per game,3 point field goal attempt per game,2 point field goal per game,...,Turn overs per game,Personal fouls per game,Points per game,Field goal percent,3 point field goal percentage,2 point field goal percentage,Free throw percentage,Effective field goal percentage,MVP,Defensive Player of the Year
0,A.C. Green,37,PF,1,1411,144,324,0,6,144,...,45,119,367,0.444444,0.000000,0.452830,0.711712,0.444444,0,0
1,A.J. Guyton,22,PG,8,630,78,192,27,69,51,...,24,35,198,0.406250,0.391304,0.414634,0.833333,0.476562,0,0
2,Aaron McKie,28,SG,33,2394,338,714,53,170,285,...,203,178,878,0.473389,0.311765,0.523897,0.768041,0.510504,0,0
3,Aaron Williams,29,PF,25,2336,297,650,0,2,297,...,132,319,838,0.456923,0.000000,0.458333,0.787097,0.456923,0,0
4,Adam Keefe,30,PF,13,836,64,159,1,3,63,...,40,102,168,0.402516,0.333333,0.403846,0.619048,0.405660,0,0


### Lets build a random forest classifier for the player data and predict MVPs

In [3]:
player_df.shape

(11083, 29)

In [4]:
player_df.dropna(inplace=True)
player_df.shape

(11083, 29)

In [5]:
player_df.dtypes

Player                                  object
Age                                      int64
Position                                object
Games started                            int64
Minutes played per game                  int64
Field goals per game                     int64
Field goals attempt per game             int64
3 point field goal per game              int64
3 point field goal attempt per game      int64
2 point field goal per game              int64
2 point field goal attempt per game      int64
Free throws per game                     int64
Free throw attempt per game              int64
Offensive rebounds per game              int64
Defensive rebounds per game              int64
Total rebounds per game                  int64
Assist per game                          int64
Steals per game                          int64
Blocks per game                          int64
Turn overs per game                      int64
Personal fouls per game                  int64
Points per ga

In [6]:
player_df

,Player,Age,Position,Games started,Minutes played per game,Field goals per game,Field goals attempt per game,3 point field goal per game,3 point field goal attempt per game,2 point field goal per game,...,Turn overs per game,Personal fouls per game,Points per game,Field goal percent,3 point field goal percentage,2 point field goal percentage,Free throw percentage,Effective field goal percentage,MVP,Defensive Player of the Year
0,A.C. Green,37,PF,1,1411,144,324,0,6,144,...,45,119,367,0.444444,0.000000,0.452830,0.711712,0.444444,0,0
1,A.J. Guyton,22,PG,8,630,78,192,27,69,51,...,24,35,198,0.406250,0.391304,0.414634,0.833333,0.476562,0,0
2,Aaron McKie,28,SG,33,2394,338,714,53,170,285,...,203,178,878,0.473389,0.311765,0.523897,0.768041,0.510504,0,0
3,Aaron Williams,29,PF,25,2336,297,650,0,2,297,...,132,319,838,0.456923,0.000000,0.458333,0.787097,0.456923,0,0
4,Adam Keefe,30,PF,13,836,64,159,1,3,63,...,40,102,168,0.402516,0.333333,0.403846,0.619048,0.405660,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11078,Zach Collins,25,C,26,1441,284,548,55,147,229,...,129,199,731,0.518248,0.374150,0.571072,0.760563,0.568431,0,0
11079,Zach LaVine,27,SG,77,2768,673,1388,204,544,469,...,194,159,1913,0.484870,0.375000,0.555687,0.848131,0.558357,0,0
11080,Zeke Nnaji,22,PF,5,728,110,196,17,65,93,...,31,105,277,0.561224,0.261538,0.709924,0.645161,0.604592,0,0
11081,Ziaire Williams,21,SF,4,561,84,196,25,97,59,...,37,58,210,0.428571,0.257732,0.595960,0.772727,0.492347,0,0


In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

X = player_df.drop(columns=['Player', 'MVP','Defensive Player of the Year','Position'])

y = player_df['MVP']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3,5,8,10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [8]:
rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)

best_rf_classifier = RandomForestClassifier(**best_params, random_state=42)

best_rf_classifier.fit(X_train, y_train)

y_pred = best_rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best Parameters: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.997593984962406


In [9]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[6634,    0],
       [  16,    0]], dtype=int64)

### Lets build a random forest classifier for the player data and predict Best Defensive player

In [10]:
rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)

best_rf_classifier = RandomForestClassifier(**best_params, random_state=42)

best_rf_classifier.fit(X_train, y_train)

y_pred = best_rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best Parameters: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.997593984962406


In [11]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[6634,    0],
       [  16,    0]], dtype=int64)

In [12]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='rbf', random_state=42) 

svm_classifier.fit(X_train, y_train)

y_pred_svm = svm_classifier.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)

conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print("Confusion Matrix:")
print(conf_matrix_svm)

SVM Accuracy: 0.997593984962406
Confusion Matrix:
[[6634    0]
 [  16    0]]


In [19]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

smote = SMOTE(sampling_strategy = 'minority',random_state=42, n_jobs=-1)
X_trian_smote, y_train_smote = smote.fit_resample(X_train,y_train)

model = RandomForestClassifier(random_state = 42)
model.fit(X_trian_smote,y_train_smote)

y_pred = model.predict(X_test)

print(accuracy_score(y_test,y_pred))

C:\Users\layto\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\base.py:363: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


0.995849873691808
